# Queries more genres and tags of US users

In [314]:
import soundcloud
import pandas as pd

client = soundcloud.Client(client_id = 'rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z')
#rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z
#2t9loNQH90kzJcsFCODdigxfp325aq4z
#cb4917402e7e92b3908cfaf84f52fe45

users_to_be_queried = list(map(int, open('users_to_be_queried.txt', 'r').read().split(', ')))
soundcloud = pd.read_csv('soundcloud.csv')
users_to_be_queried = list(set(users_to_be_queried) - set(soundcloud['user_id']))

user_ids = []
cities = []
genres_list = []
tags_list = []

for user_id in users_to_be_queried:
    favs = client.get('/users/' + str(user_id) + '/favorites')
    
    if favs:
        genres = []
        tags = []
        
        for fav in favs:
            if fav.genre:
                genres.append(fav.genre)
            if fav.tag_list:
                tags.append(fav.tag_list)
        
        user_ids.append(user_id)
        user = client.get('/users/' + str(user_id))
        cities.append(user.city)

        genres_list.append(genres)
        tags_list.append(tags)

HTTPError: 500 Server Error: Internal Server Error for url: https://api.soundcloud.com/users/167552546/favorites?client_id=rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z

In [315]:
soundcloud = pd.read_csv('soundcloud.csv')
df = pd.DataFrame({'user_id': user_ids, 'city': cities, 'genres_list': genres_list, 'tags_list': tags_list})
df = pd.concat([soundcloud, df])

df.to_csv('soundcloud.csv', index = False)

users_still_to_be_queried = list(set(users_to_be_queried) - set(df['user_id']))

file = open('users_to_be_queried.txt', 'w')
for user in users_still_to_be_queried[:-1]:
    file.write(str(user) + ", ")
file.write(str(users_still_to_be_queried[-1]))
file.close()

print(df.shape[0])
print(len(users_still_to_be_queried))
print(df.shape[0] + len(users_still_to_be_queried))

1137
1920
3057


# Finds more US users

In [294]:
import soundcloud
import pandas as pd

client = soundcloud.Client(client_id = 'rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z')
#rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z
#2t9loNQH90kzJcsFCODdigxfp325aq4z
#cb4917402e7e92b3908cfaf84f52fe45

us_users = []
next_href_exists = True

users = client.get('/users', q = 'United States', limit = 200, linked_partitioning = 1)
while (len(us_users) <= 10000) and next_href_exists:
    users = client.get(users.next_href, q = 'United States', limit = 200, linked_partitioning = 1)
    for user in users.collection:
        if user.city and ('United States' not in user.city) and (user.country == 'United States'):
            us_users.append(user.id)
    if 'next_href' not in users.keys():
        next_href_exists = False

HTTPError: 400 Client Error: Bad Request for url: https://api.soundcloud.com/users?linked_partitioning=1&client_id=rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z&offset=8200&q=United%20States&limit=200&q=United+States&limit=200&linked_partitioning=1&client_id=rTZYCxGbxmlpK8IYRpP6fWGMZXhlgp2z

In [296]:
previous_total = list(map(int, open('users_total.txt', 'r').read().split(', ')))
users_total = list(set(us_users + previous_total))

file = open('users_total.txt', 'w')
for user in users_total[:-1]:
    file.write(str(user) + ", ")
file.write(str(users_total[-1]))
file.close()

soundcloud = pd.read_csv('soundcloud.csv')
users_to_be_queried = list(set(users_total) - set(soundcloud['user_id']))

file = open('users_to_be_queried.txt', 'w')
for user in users_to_be_queried[:-1]:
    file.write(str(user) + ", ")
file.write(str(users_to_be_queried[-1]))
file.close()

print(soundcloud.shape[0])
print(len(users_to_be_queried))
print(len(users_total))

969
2088
3057
